In [0]:
CREATE SCHEMA IF NOT EXISTS genie_demo;

USE SCHEMA genie_demo;

In [0]:
-- Drop in case you’re re-running
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS sales;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS categories;
DROP TABLE IF EXISTS brands;
DROP TABLE IF EXISTS suppliers;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS stores;

-- CATEGORIES
CREATE TABLE categories (
  category_id BIGINT GENERATED ALWAYS AS IDENTITY,
  category_name STRING NOT NULL,
  CONSTRAINT pk_categories PRIMARY KEY (category_id)
)
COMMENT 'Product categories';

-- BRANDS
CREATE TABLE brands (
  brand_id BIGINT GENERATED ALWAYS AS IDENTITY,
  brand_name STRING NOT NULL,
  CONSTRAINT pk_brands PRIMARY KEY (brand_id)
)
COMMENT 'Product brands';

-- SUPPLIERS
CREATE TABLE suppliers (
  supplier_id BIGINT GENERATED ALWAYS AS IDENTITY,
  supplier_name STRING NOT NULL,
  phone STRING,
  country STRING,
  CONSTRAINT pk_suppliers PRIMARY KEY (supplier_id)
)
COMMENT 'Product suppliers';

-- PRODUCTS (FKs to categories, brands, suppliers)
CREATE TABLE products (
  product_id BIGINT GENERATED ALWAYS AS IDENTITY,
  sku STRING NOT NULL,
  product_name STRING NOT NULL,
  brand_id BIGINT NOT NULL,
  category_id BIGINT NOT NULL,
  supplier_id BIGINT NOT NULL,
  standard_cost DECIMAL(10,2) NOT NULL,
  list_price DECIMAL(10,2) NOT NULL,
  CONSTRAINT pk_products PRIMARY KEY (product_id),
  CONSTRAINT fk_products_brand FOREIGN KEY (brand_id) REFERENCES brands(brand_id),
  CONSTRAINT fk_products_category FOREIGN KEY (category_id) REFERENCES categories(category_id),
  CONSTRAINT fk_products_supplier FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
)
COMMENT 'Products with brand, category, supplier references';

-- Optional value sanity check (CHECK must be added after creation)
ALTER TABLE products ADD CONSTRAINT chk_products_price_nonneg
CHECK (standard_cost >= 0 AND list_price >= 0);

-- CUSTOMERS
CREATE TABLE customers (
  customer_id BIGINT GENERATED ALWAYS AS IDENTITY,
  first_name STRING NOT NULL,
  last_name STRING NOT NULL,
  email STRING,
  phone STRING,
  created_date DATE DEFAULT current_date(),
  CONSTRAINT pk_customers PRIMARY KEY (customer_id)
)
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported')
COMMENT 'Customers';

-- STORES
CREATE TABLE stores (
  store_id BIGINT GENERATED ALWAYS AS IDENTITY,
  store_name STRING NOT NULL,
  city STRING,
  country STRING,
  CONSTRAINT pk_stores PRIMARY KEY (store_id)
)
COMMENT 'Stores/locations';

-- EMPLOYEES (FK to stores)
CREATE TABLE employees (
  employee_id BIGINT GENERATED ALWAYS AS IDENTITY,
  first_name STRING NOT NULL,
  last_name STRING NOT NULL,
  store_id BIGINT NOT NULL,
  hire_date DATE,
  CONSTRAINT pk_employees PRIMARY KEY (employee_id),
  CONSTRAINT fk_employees_store FOREIGN KEY (store_id) REFERENCES stores(store_id)
)
COMMENT 'Employees working at stores';


In [0]:
-- SALES (line-level). No FKs by design. Partitioned by generated sale_date.
CREATE TABLE sales (
  sale_id BIGINT GENERATED ALWAYS AS IDENTITY,
  sale_ts TIMESTAMP NOT NULL,
  sale_date DATE GENERATED ALWAYS AS (CAST(sale_ts AS DATE)),
  product_id BIGINT,     -- matches products.product_id (no FK)
  customer_id BIGINT,    -- matches customers.customer_id (no FK)
  store_id BIGINT,       -- matches stores.store_id (no FK)
  employee_id BIGINT,    -- matches employees.employee_id (no FK)
  quantity INT NOT NULL,
  unit_price DECIMAL(10,2) NOT NULL,
  discount_amount DECIMAL(10,2) NOT NULL,
  CONSTRAINT pk_sales PRIMARY KEY (sale_id)
)
COMMENT 'Sales fact table (no foreign keys; keys remain same column names)';

ALTER TABLE sales ADD CONSTRAINT chk_sales_values
CHECK (quantity > 0 AND unit_price >= 0 AND discount_amount >= 0);


In [0]:
-- Lookup tables
INSERT INTO categories (category_name) VALUES
  ('Electronics'), ('Home Appliances'), ('Accessories');

INSERT INTO brands (brand_name) VALUES
  ('Acme'), ('Contoso'), ('Globex');

INSERT INTO suppliers (supplier_name, phone, country) VALUES
  ('TechSource', '+39-02-555-0001', 'Italy'),
  ('HomeGoods',  '+39-06-555-0002', 'Italy'),
  ('Gadgetron',  '+44-20-555-0003', 'UK');

-- Stores & employees
INSERT INTO stores (store_name, city, country) VALUES
  ('Milano Central', 'Milan', 'Italy'),
  ('Roma Centro',    'Rome',  'Italy'),
  ('Napoli Port',    'Naples','Italy');

INSERT INTO employees (first_name, last_name, store_id, hire_date)
SELECT 'Luca','Rossi', s.store_id, DATE'2023-05-10' FROM stores s WHERE s.store_name='Milano Central';
INSERT INTO employees (first_name, last_name, store_id, hire_date)
SELECT 'Giulia','Bianchi', s.store_id, DATE'2024-02-01' FROM stores s WHERE s.store_name='Roma Centro';
INSERT INTO employees (first_name, last_name, store_id, hire_date)
SELECT 'Marco','Verdi', s.store_id, DATE'2022-11-18' FROM stores s WHERE s.store_name='Napoli Port';

-- Customers
INSERT INTO customers (first_name, last_name, email, phone, created_date) VALUES
  ('Anna','Kowalska','anna.k@example.com','+48-555-111-222', DATE'2024-01-15'),
  ('Diego','Santoro','diego.s@example.com','+39-555-333-444', DATE'2024-03-22'),
  ('Sirin','Altun',  'sirin.a@example.com','+39-555-222-333', DATE'2025-01-09'),
  ('Andrea','Marino', 'andrea.m@example.com','+39-555-444-555', DATE'2025-02-12');

-- Products (use name lookups to resolve surrogate keys)
INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'ACMEX100','Smartphone X100', b.brand_id, c.category_id, s.supplier_id, 300.00, 499.99
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Acme' AND c.category_name='Electronics' AND s.supplier_name='TechSource';

INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'CNT-EARB','Wireless Earbuds', b.brand_id, c.category_id, s.supplier_id, 40.00, 79.99
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Contoso' AND c.category_name='Accessories' AND s.supplier_name='Gadgetron';

INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'GLB-4K55','4K TV 55\"', b.brand_id, c.category_id, s.supplier_id, 350.00, 649.99
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Globex' AND c.category_name='Electronics' AND s.supplier_name='TechSource';

INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'ACM-ESP','Espresso Machine', b.brand_id, c.category_id, s.supplier_id, 120.00, 249.90
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Acme' AND c.category_name='Home Appliances' AND s.supplier_name='HomeGoods';

INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'CNT-ROBO','Robot Vacuum', b.brand_id, c.category_id, s.supplier_id, 180.00, 329.00
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Contoso' AND c.category_name='Home Appliances' AND s.supplier_name='HomeGoods';

INSERT INTO products (sku, product_name, brand_id, category_id, supplier_id, standard_cost, list_price)
SELECT 'GLB-USBC1','USB‑C Cable 1m', b.brand_id, c.category_id, s.supplier_id, 2.00, 7.99
FROM brands b, categories c, suppliers s
WHERE b.brand_name='Globex' AND c.category_name='Accessories' AND s.supplier_name='Gadgetron';


In [0]:
-- Helper: pick employees by last name (ensures deterministic lookups)
-- Luca Rossi (Milano), Giulia Bianchi (Roma), Marco Verdi (Napoli)

-- 1) Smartphone X100 sold to Anna at Milano by Rossi
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-14 10:15:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 0.00
FROM products p
JOIN customers cu ON cu.email='anna.k@example.com'
JOIN stores st ON st.store_name='Milano Central'
JOIN employees e ON e.last_name='Rossi'
WHERE p.sku='ACMEX100';

-- 2) Wireless Earbuds to Diego in Roma by Bianchi (small discount)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-14 11:02:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       2, p.list_price, 5.00
FROM products p
JOIN customers cu ON cu.email='diego.s@example.com'
JOIN stores st ON st.store_name='Roma Centro'
JOIN employees e ON e.last_name='Bianchi'
WHERE p.sku='CNT-EARB';

-- 3) 4K TV to Andrea in Milano (Rossi)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-15 16:30:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 50.00
FROM products p
JOIN customers cu ON cu.email='andrea.m@example.com'
JOIN stores st ON st.store_name='Milano Central'
JOIN employees e ON e.last_name='Rossi'
WHERE p.sku='GLB-4K55';

-- 4) Espresso Machine to Sirin in Roma (Bianchi)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-15 09:05:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 10.00
FROM products p
JOIN customers cu ON cu.email='sirin.a@example.com'
JOIN stores st ON st.store_name='Roma Centro'
JOIN employees e ON e.last_name='Bianchi'
WHERE p.sku='ACM-ESP';

-- 5) Robot Vacuum to Diego in Napoli (Verdi)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-15 12:20:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 0.00
FROM products p
JOIN customers cu ON cu.email='diego.s@example.com'
JOIN stores st ON st.store_name='Napoli Port'
JOIN employees e ON e.last_name='Verdi'
WHERE p.sku='CNT-ROBO';

-- 6) USB‑C Cable to Anna in Milano (Rossi), qty 3
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-16 08:50:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       3, p.list_price, 0.00
FROM products p
JOIN customers cu ON cu.email='anna.k@example.com'
JOIN stores st ON st.store_name='Milano Central'
JOIN employees e ON e.last_name='Rossi'
WHERE p.sku='GLB-USBC1';

-- 7) Earbuds to Andrea in Roma (Bianchi)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-16 13:10:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 0.00
FROM products p
JOIN customers cu ON cu.email='andrea.m@example.com'
JOIN stores st ON st.store_name='Roma Centro'
JOIN employees e ON e.last_name='Bianchi'
WHERE p.sku='CNT-EARB';

-- 8) Espresso Machine to Anna in Milano (Rossi)
INSERT INTO sales (sale_ts, product_id, customer_id, store_id, employee_id, quantity, unit_price, discount_amount)
SELECT TIMESTAMP '2025-09-16 18:42:00', p.product_id, cu.customer_id, st.store_id, e.employee_id,
       1, p.list_price, 15.00
FROM products p
JOIN customers cu ON cu.email='anna.k@example.com'
JOIN stores st ON st.store_name='Milano Central'
JOIN employees e ON e.last_name='Rossi'
WHERE p.sku='ACM-ESP';
